<a href="https://colab.research.google.com/github/ebelleng/EDDA_desafio1/blob/main/Copy_of_Packing_squares_algoritms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packing Squares 


## Estados


In [ ]:
global squares
squares = 8

tamano_cuadraos = [3,2,2,2,1,1,1,1]

In [ ]:
class Estado:
  X = [0] * squares
  Y = [0] * squares

  def __init__(self, i=0, x=0, y=0):
    self.X[i] = x
    self.Y[i] = y

In [ ]:
# [3,2,2,2,1,1,1,1]
#X = [](1, 1, 3, 4, 4, 5, 5, 5)
#Y = [](1, 4, 4, 1, 3, 3, 4, 5)

In [ ]:
class Accion:
  def __init__(self, i, x, y):
    self.i = i
    self.x = x
    self.y = y

In [ ]:
def transicion(estado, accion):
  estado_siguiente = State(accion.i, accion.x, accion.y)
  return estado_siguiente

In [ ]:
def obtener_acciones(state, i): ## Etienne del futuro: el i es la posicion que se maneja en el main
  lista_acciones_posibles = []

  

  
  return

In [ ]:
# i = 0 x=1 y=1
#X = [](1, 0, 0, 0, 0, 0, 0, 0)
#Y = [](1, 0, 0, 0, 0, 0, 0, 0)

In [ ]:
## ESTADO 0
# i = 0 x=0 y=0
#X = [](0, 0, 0, 0, 0, 0, 0, 0)
#Y = [](0, 0, 0, 0, 0, 0, 0, 0)

Action -> i, x,y

transition(state, action a){
    state_next = State(a.i, a.x, a.y)
    return state_next
}


In [1]:
lista = [3,2,2,2,1,1,1,1]
set(lista)

{1, 2, 3}

In [ ]:
def comprarEstados(): # Heuristica
  ## algoritmo 

  return State()